## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details.

In [235]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict


In [236]:
# More imports
from nltk.corpus import stopwords
from string import punctuation

import re
import emoji

In [56]:
sw = stopwords.words("english")
punctuation = set(punctuation)

In [259]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [271]:
convention_data = []


# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

# query text and part from .db
query_results = convention_cur.execute(
                            '''
                            SELECT text, party FROM conventions;
                            ''')

for row in query_results :
    # removing some difficult character from the text manually + lower
    l = row[0].lower().replace('«','').replace('›','').replace('”', '').replace("’", '').replace("“",'')

    # lower party
    p = row[1].lower()

    # removing urls
    ls = re.sub(r'http\S+', '', l)

    # remove punctuation and split
    lss = "".join([i for i in ls if i not in punctuation]).split()
    # remove stop words
    lps = [i for i in lss if i not in set(sw)]
    # remove any emojis in text
    lpss = [i for i in lps if emoji.is_emoji(i) != True]
    

    # append both to list
    convention_data.append([' '.join(lpss), p])



In [272]:
print(convention_data[0])

['skip content company careers press freelancers blog × services transcription captions foreign subtitles translation freelancers contact login return transcript library home transcript categories transcripts 2020 election transcripts classic speech transcripts congressional testimony hearing transcripts debate transcripts donald trump transcripts entertainment transcripts financial transcripts interview transcripts political transcripts press conference transcripts speech transcripts sports transcripts technology transcripts aug 21 2020 2020 democratic national convention dnc night 4 transcript rev blog transcripts 2020 election transcripts 2020 democratic national convention dnc night 4 transcript night 4 2020 democratic national convention dnc august 20 read full transcript event transcribe content try rev free save time transcribing captioning subtitling', 'democratic']


Let's look at some random entries and see if they look right. 

In [273]:
random.choices(convention_data,k=10)

[['could taken away doesnt win', 'republican'],
 ['bernie believes healthcare human right contingent job knows rebuild crumbling infrastructure creating millions goodpaying union jobs combating climate change bernies moral clarity emboldened democratic partys fight justice grassroots energy supporters amended important advances platform bernie continue lead movement helps defeat trump delivers transformational change im excited place nomination name great champion working class senator bernie sanders',
  'democratic'],
 ['thats good thank much appreciate sudha sundari narayanan phenomenal success born india came united states 13 years ago sudha talented software developer husband raising two beautiful wonderful children apples life right',
  'republican'],
 ['weve made history along way', 'democratic'],
 ['got back', 'republican'],
 ['small indiana town foundation faith jesus christ laid conviction sprung love people service others church service mike met love life karen married three 

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [274]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2382 as features in the model.


In [276]:
def conv_features(text,fw):
     ret_dict = dict()
     
     for i in text.split():
          if i in fw:
               ret_dict[i] = True
     
          
    
    
     return(ret_dict)

In [277]:
conv_features("donald is the president", feature_words)

{'donald': True, 'president': True}

In [278]:
conv_features("people are american in america",feature_words)

{'people': True, 'american': True, 'america': True}

In [279]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [280]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [281]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [282]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.498


In [283]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           republ : democr =     25.8 : 1.0
                   votes = True           democr : republ =     23.8 : 1.0
             enforcement = True           republ : democr =     21.5 : 1.0
                 destroy = True           republ : democr =     19.2 : 1.0
                freedoms = True           republ : democr =     18.2 : 1.0
                 climate = True           democr : republ =     17.8 : 1.0
                supports = True           republ : democr =     17.1 : 1.0
                   crime = True           republ : democr =     16.1 : 1.0
                   media = True           republ : democr =     14.9 : 1.0
                 beliefs = True           republ : democr =     13.0 : 1.0
               countries = True           republ : democr =     13.0 : 1.0
                 defense = True           republ : democr =     13.0 : 1.0
                  defund = True           republ : democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

Considering Covid and the way that was sensationalized by the Republican party as the "China Flu" it is unsurprising that it is one of the strongest indicators of the party. In fact many of the Republican party indicators stem from buzzwords within the populus that are rooted from news sources like Fox News. It is also unsuprising that Democrats most key words are votes and climate as going out and voting was key in the most recent presidential election campaigning and the current climate change issues are at the forefront of the party's, at least, campaign policies. I don't see anything that particularly subverts my expectations for either party. 

In terms of overall accuracy, the model is at about 50/50 on the test set, and this is likely due to overpredicting republicans as they have more 'informative' words within the model



## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [284]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [285]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [286]:
tweet_data = []

for row in results:
    # decode to utf8 then lower then remove some stubborn characters
    l = row[2].decode('utf8').lower().replace('”', '').replace("’", '').replace("“",'')
    # remove urls
    ls = re.sub(r'http\S+', '', l)
    p = row[1].lower()
    n = row[0].lower()
    # remove punctuation
    lp = "".join([ch for ch in ls if ch not in punctuation]).split()
    # remove stopwords
    lps = [i for i in lp if i not in set(sw)]
    # remove emojis
    lpss = [i for i in lps if emoji.is_emoji(i) != True]
    

    # create list
    tweet_data.append([' '.join(lpss), p])
# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [287]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [288]:

for tweet, party in tweet_data_sample :
    converted = conv_features(tweet,feature_words)
    estimated_party = classifier.classify(converted)
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: earlier today spoke house floor abt protecting health care women praised ppmarmonte work central coast
Actual party is democratic and our classifer says republican.

Here's our (cleaned) tweet: go tribe rallytogether
Actual party is democratic and our classifer says democratic.

Here's our (cleaned) tweet: apparently trump thinks easy students overwhelmed crushing burden debt pay student loans trumpbudget
Actual party is democratic and our classifer says republican.

Here's our (cleaned) tweet: grateful first responders rescue personnel firefighters police volunteers working tirelessly keep people safe provide muchneeded help putting lives line
Actual party is republican and our classifer says republican.

Here's our (cleaned) tweet: lets make even greater kag
Actual party is republican and our classifer says republican.

Here's our (cleaned) tweet: 1hr cavs tie series 22 im allin216 repbarbaralee scared roadtovictory
Actual party is democratic and our class

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [298]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['republican','democratic']
results = defaultdict(lambda: defaultdict(int))
for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
    converted = conv_features(tweet,feature_words)
    
    # get the estimated party
    estimated_party = classifier.classify(converted)
    prob = classifier.prob_classify(converted)
    
    results[party][estimated_party] += 1
    if idx > num_to_score : 
        break

In [299]:
results

defaultdict(<function __main__.<lambda>()>,
            {'republican': defaultdict(int,
                         {'republican': 3777, 'democratic': 605}),
             'democratic': defaultdict(int,
                         {'republican': 4708, 'democratic': 912})})

In [300]:
print("Classifier Accuracy Score:", str(round((results['republican']['republican'] + results['democratic']['democratic']) / (results['republican']['republican'] + results['democratic']['democratic'] + results['democratic']['republican'] + results['republican']['democratic'])*100, 2))+"%")
print("")
print("Classifier Precision Score:", str(round((results['republican']['republican']) / (results['republican']['republican'] + results['republican']['democratic'])*100, 2)) + "%")
print("")
print("Classifier Recall Score:", str(round((results['democratic']['democratic']) / (results['democratic']['democratic'] + results['democratic']['republican'])*100, 2))+ "%")

Classifier Accuracy Score: 46.88%

Classifier Precision Score: 86.19%

Classifier Recall Score: 16.23%


### Reflections

Looking at how the classifier performed on the new data, the results are not great. The model is overtuned for the republican party causing a high precision and low recall to then create a very low overall accuracy. Tweets can be very hard to get a proper read on as sarcasm or quoting the other party does not translate to a model like this. Due to that, there can lead to overpredictions of one class. Additionally, there was not many Democratic feature words up above compared to Republican and this could lead to the overprediction as well. 